#**Sistem Pendukung Keputusan Kredit Menggunakan Algoritma KNN**

#**Kelompok 03:**
- MUhammad Veka Syahputra (4612422022)
- ZAMACHSYAFI SHIDQI ATHALLAH (4612422025)
- Fauzan Dwi Eryawan (4612422037)
- MOCHAMAD RIZQI RAHCMAN (4612422038)
- Agus Sutiyanto (4612422052)


##**KNN (K-Nearest Neighbors) & Ensemble Bagging**

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

# Bobot untuk setiap opsi kriteria
bobot_k1 = np.array([0.10, 0.15, 0.20, 0.25, 0.30])
bobot_k2 = np.array([0.10, 0.15, 0.20, 0.25, 0.30])
bobot_k3 = np.array([0.10, 0.15, 0.20, 0.25, 0.30])
bobot_k4 = np.array([0.10, 0.15, 0.20, 0.25, 0.30])
bobot_k5 = np.array([0.10, 0.15, 0.20, 0.25, 0.30])

dataset_train = pd.read_excel('data_training.xlsx')
dataset_test = pd.read_excel('data_testing.xlsx')

# Input data dari pengguna
input_user = []
for i in range(5):
    input_user.append(int(input("Masukkan nilai untuk K{}: ".format(i+1))))


jarak_list = []
hasil_list = []  
for index, row in dataset_train.iterrows():
    data_train = row[:-1].values  
    hasil_list.append(row['Hasil']) 
    jarak = np.sqrt(
        np.sum([
            bobot_k1[input_user[0]-1] * ((input_user[0] - data_train[0]) ** 2),
            bobot_k2[input_user[1]-1] * ((input_user[1] - data_train[1]) ** 2),
            bobot_k3[input_user[2]-1] * ((input_user[2] - data_train[2]) ** 2),
            bobot_k4[input_user[3]-1] * ((input_user[3] - data_train[3]) ** 2),
            bobot_k5[input_user[4]-1] * ((input_user[4] - data_train[4]) ** 2)
        ])
    )
    jarak_list.append((index, jarak))

# Mengurutkan berdasarkan jarak
jarak_list_sorted = sorted(jarak_list, key=lambda x: x[1])

# 9 tetangga terdekat
neighbors = jarak_list_sorted[:9]

nearest_neighbors_data = []
for index, jarak in neighbors:
    nearest_neighbors_data.append({
        'K1': dataset_train.iloc[index]['K1'],
        'K2': dataset_train.iloc[index]['K2'],
        'K3': dataset_train.iloc[index]['K3'],
        'K4': dataset_train.iloc[index]['K4'],
        'K5': dataset_train.iloc[index]['K5'],
        'Hasil': dataset_train.iloc[index]['Hasil'],
        'Jarak': jarak
    })
nearest_neighbors = pd.DataFrame(nearest_neighbors_data)
acceptance_prob = nearest_neighbors['Hasil'].mean() * 100

# Menampilkan hasil
print("Probabilitas diterima: {:.2f}%".format(acceptance_prob))

if acceptance_prob > 50:
    print("Memenuhi Kelayakan")
else :
    print("Tidak Memenuhi")

print("\nHasil Kedekatan 9 Tetangga Terdekat (Diurutkan Berdasarkan Jarak):")
print(nearest_neighbors)


X_train = dataset_train.iloc[:, :-1]
y_train = dataset_train.iloc[:, -1]

# Bagging
from sklearn.ensemble import BaggingClassifier
bagging_classifier = BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=9), n_estimators=10, random_state=42)

bagging_classifier.fit(X_train, y_train)

X_test = dataset_test.iloc[:, :-1]
y_test = dataset_test.iloc[:, -1]

predictions = bagging_classifier.predict(X_test)

# akurasi
accuracy = accuracy_score(y_test, predictions)
print("\nAkurasi", accuracy)

# confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)
print("\nConfusion matrix:\n", conf_matrix)

Masukkan nilai untuk K1: 5
Masukkan nilai untuk K2: 3
Masukkan nilai untuk K3: 4
Masukkan nilai untuk K4: 3
Masukkan nilai untuk K5: 4
Probabilitas diterima: 88.89%
Memenuhi Kelayakan

Hasil Kedekatan 9 Tetangga Terdekat (Diurutkan Berdasarkan Jarak):
   K1  K2  K3  K4  K5  Hasil     Jarak
0   5   3   4   3   4      1  0.000000
1   5   2   4   3   4      1  0.447214
2   5   3   4   2   4      0  0.447214
3   5   3   4   4   4      1  0.447214
4   5   4   4   3   4      1  0.447214
5   5   3   3   3   4      1  0.500000
6   5   3   4   3   3      1  0.500000
7   5   3   4   3   5      1  0.500000
8   5   3   5   3   4      1  0.500000

Akurasi 0.9968051118210862

Confusion matrix:
 [[233   0]
 [  1  79]]
